# "Optimal value extraction by M/L"

## Two objects should be considered !!!

### Use seaborn for pairplot
!pip install -q seaborn

### Use some functions from tensorflow_docs
!pip install -q git+https://github.com/tensorflow/docs

## This example uses the tf.keras API, see this guide for details.

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

import pathlib

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

In [ ]:
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

print(tf.__version__)

In [ ]:
import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

# make dataset for sPGD result

In [ ]:
dataset_org = pd.read_csv(r'F:\CON_2019_HMC_PGD-MINESET\Second_model\Case2\whe_ro_to_send\New_Testing_parameters.csv')

In [ ]:
dataset = dataset_org
column_names = dataset.columns
dataset = dataset.loc[:,column_names]
dataset.head()

# Dataset for "Area"
1. pop "Pelvic"

In [ ]:
dataset.pop("Pelvic")
dataset.head()

In [ ]:
train_dataset = dataset.sample(frac=0.8,random_state=0)
test_dataset = dataset.drop(train_dataset.index)	

fig = sns.pairplot(train_dataset[["HLAYER01", "HLAYER02", "HLAYER03","HLAYER04","HLAYER05","Area"]], diag_kind="kde")	
fig.savefig('foo.png')

In [ ]:
train_stats = train_dataset.describe()
train_stats.pop("Area")
train_stats = train_stats.transpose()
train_stats.tail()

In [ ]:
train_labels = train_dataset.pop('Area')
test_labels = test_dataset.pop('Area')

In [ ]:

def norm(x):
  return (x - train_stats['mean']) / train_stats['std']
normed_train_data = norm(train_dataset)
normed_test_data  = norm(test_dataset)

normed_train_data.tail()

In [ ]:
def build_model():
    model = keras.Sequential([
      layers.Dense(128, activation='relu', kernel_regularizer=keras.regularizers.l2(0.001), input_shape=[len(train_dataset.keys())]),
      layers.Dropout(0.5),
      layers.Dense(128, activation='relu', kernel_regularizer=keras.regularizers.l2(0.001)), 
      layers.Dropout(0.5),
      layers.Dense(1)
      ])

    optimizer = tf.keras.optimizers.RMSprop(0.01)
    # optimizer='sgd'
    
    model.compile(
      loss='mse',
      optimizer=optimizer,
      metrics=['mae', 'mse', 'mape', 'accuracy'])

    return model

model_Area = build_model()
model_Area.summary()

In [ ]:
# example_batch  = normed_train_data[:10]
example_batch  = normed_train_data[:10]
example_result = model_Area.predict(example_batch)
example_result

In [ ]:
# export trained model
import os
current_dir = os.getcwd()
print(current_dir)
model_Area.save("{}/trainedMODEL_Area.h5".format(current_dir))

In [ ]:
EPOCHS = 1000

history = model_Area.fit(
  normed_train_data, train_labels,
  epochs=EPOCHS, validation_split = 0.2, verbose=0,
  callbacks=[tfdocs.modeling.EpochDots()])


## MAE refers to Mean Absolute Error
## MAPE refers to Mean Absolute Percentage Error
## MSE refers to Mean Squared Error

In [ ]:
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist.tail()


In [ ]:
#%matplotlib inline
import matplotlib.pyplot as plt


fig, loss_ax = plt.subplots()

acc_ax = loss_ax.twinx()

loss_ax.plot(history.history['loss'], 'y', label='train loss')
loss_ax.plot(history.history['val_loss'], 'r', label='val loss')

acc_ax.plot(history.history['accuracy'], 'b', label='train acc')
acc_ax.plot(history.history['val_accuracy'], 'g', label='val acc')

loss_ax.set_xlabel('epoch')
loss_ax.set_ylabel('loss')
acc_ax.set_ylabel('accuray')

loss_ax.legend(loc='upper left')
acc_ax.legend(loc='lower right')

plt.show()

In [ ]:
# plotter = tfdocs.plots.HistoryPlotter(smoothing_std=2)
# plotter.plot({'Basic': history}, metric = "mae")
# plt.ylim([0, 10])
# plt.ylabel('MAE [MIN_AREA]')
def plot_history(history):
  hist = pd.DataFrame(history.history)
  hist['epoch'] = history.epoch

  plt.figure(figsize=(8,12))

  plt.subplot(2,1,1)
  plt.xlabel('Epoch')
  plt.ylabel('Mean Abs Error [Area]')
  plt.plot(hist['epoch'], hist['mae'],
           label='Train Error')
  plt.plot(hist['epoch'], hist['val_mae'],
           label = 'Val Error')
  plt.ylim([0,100])
  plt.legend()

  plt.subplot(2,1,2)
  plt.xlabel('Epoch')
  plt.ylabel('Mean Square Error [$Area^2$]')
  plt.plot(hist['epoch'], hist['mse'],
           label='Train Error')
  plt.plot(hist['epoch'], hist['val_mse'],
           label = 'Val Error')
  plt.ylim([0,20])
  plt.legend()
  plt.show()

plot_history(history)


In [ ]:
plotter.plot({'Basic': history}, metric = "mae")
plt.ylim([0, 5000])
plt.ylabel('MAE [MIN_AREA]')


In [ ]:
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

early_history = model_Area.fit(normed_train_data, train_labels, 
                    epochs=EPOCHS, validation_split = 0.2, verbose=0, 
                    callbacks=[early_stop, tfdocs.modeling.EpochDots()])


In [ ]:
plotter.plot({'Early Stopping': early_history}, metric = "mape")
plt.ylim([0, 1])
plt.ylabel('MAPE [Area]')



In [ ]:
loss, mae, mse, mape = model_Area.evaluate(normed_test_data, test_labels, verbose=2)
print("Testing set Mean Abs Error: {:5.2f} Area".format(mae))

# predictions by using trained model

In [ ]:
# predictions by trained model
test_predictions = model_Area.predict(normed_test_data).flatten()

f = open(r'F:\CON_2019_HMC_PGD-MINESET\Second_model\Case2\whe_ro_to_send\Prediction_Area.csv','w')
for i in range(0,len(test_predictions)):
    #print(test_predictions[i])
    f.write(str(test_predictions[i])+',')
f.close()
#print(test_predictions)


In [ ]:
a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True Values [MIN_AREA]')
plt.ylabel('Predictions [MIN_AREA]')
lims = [0, 9000]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims, lims)	

In [ ]:
error = test_predictions - test_labels
plt.hist(error, bins = 25)
plt.xlabel("Prediction Error [MIN_AREA]")
_ = plt.ylabel("Count")	

# Dataset for "Pelvic displacement"
1. pop "Area"


In [ ]:
dataset = dataset_org
column_names = dataset.columns
dataset = dataset.loc[:,column_names]
dataset.head()

In [ ]:
dataset.pop("Area")
dataset.head()

In [ ]:
train_dataset = dataset.sample(frac=0.8,random_state=0)
test_dataset = dataset.drop(train_dataset.index)	

# fig = sns.pairplot(train_dataset[["PARTNO19", "PARTNO29", "PARTNO28","PARTNO10","PARTNO17","Area_failure"]], diag_kind="kde")	
# fig.savefig('foo.png')

In [ ]:
train_stats = train_dataset.describe()
train_stats.pop("Pelvic")
train_stats = train_stats.transpose()
train_stats

In [ ]:
train_labels = train_dataset.pop('Pelvic')
test_labels = test_dataset.pop('Pelvic')

In [ ]:
# def norm(x):
#   return (x - train_stats['mean']) / train_stats['std']
normed_train_data = norm(train_dataset)
normed_test_data = norm(test_dataset)

In [ ]:
# def build_model():
#   model = keras.Sequential([
#     layers.Dense(64, activation='relu', input_shape=[len(train_dataset.keys())]),
#     layers.Dense(64, activation='relu'),
#     layers.Dense(1)
#   ])
#   optimizer = tf.keras.optimizers.RMSprop(0.001)
#   model.compile(loss='mse',
#                 optimizer=optimizer,
#                 metrics=['mae', 'mse', 'mape'])
#   return model
model_Pelvic = build_model()
model_Pelvic.summary()

In [ ]:
# export trained model
# import os
# current_dir = os.getcwd()
# print(current_dir)
model_Area.save("{}/trainedMODEL_Pelvic.h5".format(current_dir))

In [ ]:
EPOCHS = 1000

history = model_Area.fit(
  normed_train_data, train_labels,
  epochs=EPOCHS, validation_split = 0.2, verbose=0,
  callbacks=[tfdocs.modeling.EpochDots()])

In [ ]:
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist.tail()

In [ ]:
plotter = tfdocs.plots.HistoryPlotter(smoothing_std=2)
plotter.plot({'Basic': history}, metric = "mape")
plt.ylim([0, 100])
plt.ylabel('MAPE [Pelvic Disp]')

In [ ]:
plotter.plot({'Basic': history}, metric = "mae")
plt.ylim([0, 5000])
plt.ylabel('MAE [Pelvic Disp]')

In [ ]:
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

early_history = model_Area.fit(normed_train_data, train_labels, 
                    epochs=EPOCHS, validation_split = 0.2, verbose=0, 
                    callbacks=[early_stop, tfdocs.modeling.EpochDots()])

In [ ]:
plotter.plot({'Early Stopping': early_history}, metric = "mape")
plt.ylim([0, 1])
plt.ylabel('MAPE [Pelvic Disp]')

In [ ]:
loss, mae, mse, mape = model_Area.evaluate(normed_test_data, test_labels, verbose=2)
print("Testing set Mean Abs Error: {:5.2f} Pelvic Disp".format(mae))

In [ ]:
# predictions by trained model
test_predictions = model_Area.predict(normed_test_data).flatten()

f = open(r'F:\CON_2019_HMC_PGD-MINESET\Second_model\Case2\whe_ro_to_send\Prediction_Pelvic.csv','w')
for i in range(0,len(test_predictions)):
    #print(test_predictions[i])
    f.write(str(test_predictions[i])+',')
f.close()
#print(test_predictions)


In [ ]:
a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True Values [Pelvic Disp]')
plt.ylabel('Predictions [Pelvic Disp]')
lims = [0, 150]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims, lims)	

In [ ]:
error = test_predictions - test_labels
plt.hist(error, bins = 25)
plt.xlabel("Prediction Error [Pelvic Disp]")
_ = plt.ylabel("Count")	

# End Dataset for "Pelvic displacement"

# Generic Algorithm

In [ ]:
import pandas as pd
import numpy as np
import os
from tensorflow.keras.models import Sequential, load_model
from scipy import optimize
import time


In [ ]:
# Number of Curve points
point = 2480
# Number of variables 
var = 56


In [ ]:
# # Get current directory path
# current_dir = os.path.dirname(os.path.realpath(__file__))


# # Import training raw data
# T_dataset = pd.read_csv('{}/Thorax/new_training_data_thorax2_mix3_final_200612.csv'.format(current_dir))
# Tdata = T_dataset.iloc[:,1:var+point+1]
# # Classification of cross-validation data 
# T_TRAIN = Tdata.sample(frac=0.80, random_state=0)
# T_TEST = Tdata.drop(T_TRAIN.index)

# T_TRAIN = T_TRAIN.values
# T_TEST = T_TEST.values


In [ ]:
# predict_func
# Mean square error between test and prediction

#column_names = dataset.columns
#dataset = dataset.loc[:,column_names]
# def norm(x):
#   return (x - train_stats['mean']) / train_stats['std']
# normed_train_data = norm(train_dataset)
# normed_test_data = norm(test_dataset)

def predict_func(column_names):

    min_Area   = -3478.3
    max_Area   = 3849.076
    min_Pelvic = 67.74186
    max_Pelvic = 82.59047

    PARTNO01 = column_names[0]
    PARTNO02 = column_names[1]
    PARTNO03 = column_names[2]
    PARTNO04 = column_names[3]
    PARTNO05 = column_names[4]
    PARTNO06 = column_names[5]
    PARTNO07 = column_names[6]
    PARTNO08 = column_names[7]
    PARTNO09 = column_names[8]
    PARTNO10 = column_names[9]

    PARTNO11 = column_names[10]
    PARTNO12 = column_names[11]
    PARTNO13 = column_names[12]
    PARTNO14 = column_names[13]
    PARTNO15 = column_names[14]
    PARTNO16 = column_names[15]
    PARTNO17 = column_names[16]
    PARTNO18 = column_names[17]
    PARTNO19 = column_names[18]
    PARTNO20 = column_names[19]

    PARTNO21 = column_names[20]
    PARTNO22 = column_names[21]
    PARTNO23 = column_names[22]
    PARTNO24 = column_names[23]
    PARTNO25 = column_names[24]
    PARTNO26 = column_names[25]
    PARTNO27 = column_names[26]
    PARTNO28 = column_names[27]
    PARTNO29 = column_names[28]
    PARTNO30 = column_names[29]

    PARTNO31 = column_names[30]
    PARTNO32 = column_names[31]
    PARTNO33 = column_names[32]
    PARTNO34 = column_names[33]
    PARTNO35 = column_names[34]
    PARTNO36 = column_names[35]
    PARTNO37 = column_names[36]
    PARTNO38 = column_names[37]
    PARTNO39 = column_names[38]
    PARTNO40 = column_names[39]

    PARTNO41 = column_names[40]
    PARTNO42 = column_names[41]
    PARTNO43 = column_names[42]
    PARTNO44 = column_names[43]
    PARTNO45 = column_names[44]

    PARTNO46 = column_names[45]
    PARTNO47 = column_names[46]
    PARTNO48 = column_names[47]
    PARTNO49 = column_names[48]
    PARTNO50 = column_names[49]

    PARTNO51 = column_names[50]
    PARTNO52 = column_names[51]
    PARTNO53 = column_names[52]
    PARTNO54 = column_names[53]
    PARTNO55 = column_names[54]
    PARTNO56 = column_names[55]

    # Prediction data standardize
    # Common variables
    PARTNO01N = norm(PARTNO01)
    PARTNO02N = norm(PARTNO02)
    PARTNO03N = norm(PARTNO03)
    PARTNO04N = norm(PARTNO04) 
    PARTNO05N = norm(PARTNO05)
    PARTNO06N = norm(PARTNO06)
    PARTNO07N = norm(PARTNO07)
    PARTNO08N = norm(PARTNO08)
    PARTNO09N = norm(PARTNO09) 
    PARTNO10N = norm(PARTNO10)

    PARTNO11N = norm(PARTNO11)
    PARTNO12N = norm(PARTNO12)
    PARTNO13N = norm(PARTNO13)
    PARTNO14N = norm(PARTNO14) 
    PARTNO15N = norm(PARTNO15)
    PARTNO16N = norm(PARTNO16)
    PARTNO17N = norm(PARTNO17)
    PARTNO18N = norm(PARTNO18)
    PARTNO19N = norm(PARTNO19) 
    PARTNO20N = norm(PARTNO20)

    PARTNO21N = norm(PARTNO21)
    PARTNO22N = norm(PARTNO22)
    PARTNO23N = norm(PARTNO23)
    PARTNO24N = norm(PARTNO24) 
    PARTNO25N = norm(PARTNO25)
    PARTNO26N = norm(PARTNO26)
    PARTNO27N = norm(PARTNO27)
    PARTNO28N = norm(PARTNO28)
    PARTNO29N = norm(PARTNO29) 
    PARTNO30N = norm(PARTNO30)

    PARTNO31N = norm(PARTNO31)
    PARTNO32N = norm(PARTNO32)
    PARTNO33N = norm(PARTNO33)
    PARTNO34N = norm(PARTNO34) 
    PARTNO35N = norm(PARTNO35)
    PARTNO36N = norm(PARTNO36)
    PARTNO37N = norm(PARTNO37)
    PARTNO38N = norm(PARTNO38)
    PARTNO39N = norm(PARTNO39) 
    PARTNO40N = norm(PARTNO40)

    PARTNO41N = norm(PARTNO41)
    PARTNO42N = norm(PARTNO42)
    PARTNO43N = norm(PARTNO43)
    PARTNO44N = norm(PARTNO44) 
    PARTNO45N = norm(PARTNO45)
    PARTNO46N = norm(PARTNO46)

    PARTNO47N = norm(PARTNO47)
    PARTNO48N = norm(PARTNO48)
    PARTNO49N = norm(PARTNO49)
    PARTNO50N = norm(PARTNO50) 
    PARTNO51N = norm(PARTNO51)
    PARTNO52N = norm(PARTNO52)
    PARTNO53N = norm(PARTNO53)
    PARTNO54N = norm(PARTNO54)
    PARTNO55N = norm(PARTNO55) 
    PARTNO56N = norm(PARTNO56)    

    # # Result of prediction data   
    # predict_cur_Arer = model_Area.predict([[
    #     PARTNO01N,        PARTNO02N,        PARTNO03N,        PARTNO04N,        PARTNO05N, \
    #     PARTNO06N,        PARTNO07N,        PARTNO08N,        PARTNO09N,        PARTNO10N, \
    #     PARTNO11N,        PARTNO12N,        PARTNO13N,        PARTNO14N,        PARTNO15N, \
    #     PARTNO16N,        PARTNO17N,        PARTNO18N,        PARTNO19N,        PARTNO20N, \
    #     PARTNO21N,        PARTNO22N,        PARTNO23N,        PARTNO24N,        PARTNO25N, \
    #     PARTNO26N,        PARTNO27N,        PARTNO28N,        PARTNO29N,        PARTNO30N, \
    #     PARTNO31N,        PARTNO32N,        PARTNO33N,        PARTNO34N,        PARTNO35N, \
    #     PARTNO36N,        PARTNO37N,        PARTNO38N,        PARTNO39N,        PARTNO40N, \
    #     PARTNO41N,        PARTNO42N,        PARTNO43N,        PARTNO44N,        PARTNO45N, \
    #     PARTNO46N,        PARTNO47N,        PARTNO48N,        PARTNO49N,        PARTNO50N, \
    #     PARTNO51N,        PARTNO52N,        PARTNO53N,        PARTNO54N,        PARTNO55N, \
    #     PARTNO56N
    #     ]])[0]
    # predict_cur_Pelvic = model_Pelvic.predict([[
    #     PARTNO01N,        PARTNO02N,        PARTNO03N,        PARTNO04N,        PARTNO05N, \
    #     PARTNO06N,        PARTNO07N,        PARTNO08N,        PARTNO09N,        PARTNO10N, \
    #     PARTNO11N,        PARTNO12N,        PARTNO13N,        PARTNO14N,        PARTNO15N, \
    #     PARTNO16N,        PARTNO17N,        PARTNO18N,        PARTNO19N,        PARTNO20N, \
    #     PARTNO21N,        PARTNO22N,        PARTNO23N,        PARTNO24N,        PARTNO25N, \
    #     PARTNO26N,        PARTNO27N,        PARTNO28N,        PARTNO29N,        PARTNO30N, \
    #     PARTNO31N,        PARTNO32N,        PARTNO33N,        PARTNO34N,        PARTNO35N, \
    #     PARTNO36N,        PARTNO37N,        PARTNO38N,        PARTNO39N,        PARTNO40N, \
    #     PARTNO41N,        PARTNO42N,        PARTNO43N,        PARTNO44N,        PARTNO45N, \
    #     PARTNO46N,        PARTNO47N,        PARTNO48N,        PARTNO49N,        PARTNO50N, \
    #     PARTNO51N,        PARTNO52N,        PARTNO53N,        PARTNO54N,        PARTNO55N, \
    #     PARTNO56N
    #     ]])[0]        
 
     # Result of prediction data   
    predict_cur_Area = model_Area.predict([[
        PARTNO01,        PARTNO02,        PARTNO03,        PARTNO04,        PARTNO05, \
        PARTNO06,        PARTNO07,        PARTNO08,        PARTNO09,        PARTNO10, \
        PARTNO11,        PARTNO12,        PARTNO13,        PARTNO14,        PARTNO15, \
        PARTNO16,        PARTNO17,        PARTNO18,        PARTNO19,        PARTNO20, \
        PARTNO21,        PARTNO22,        PARTNO23,        PARTNO24,        PARTNO25, \
        PARTNO26,        PARTNO27,        PARTNO28,        PARTNO29,        PARTNO30, \
        PARTNO31,        PARTNO32,        PARTNO33,        PARTNO34,        PARTNO35, \
        PARTNO36,        PARTNO37,        PARTNO38,        PARTNO39,        PARTNO40, \
        PARTNO41,        PARTNO42,        PARTNO43,        PARTNO44,        PARTNO45, \
        PARTNO46,        PARTNO47,        PARTNO48,        PARTNO49,        PARTNO50, \
        PARTNO51,        PARTNO52,        PARTNO53,        PARTNO54,        PARTNO55, \
        PARTNO56
        ]])[0]
    predict_cur_Pelvic = model_Pelvic.predict([[
        PARTNO01,        PARTNO02,        PARTNO03,        PARTNO04,        PARTNO05, \
        PARTNO06,        PARTNO07,        PARTNO08,        PARTNO09,        PARTNO10, \
        PARTNO11,        PARTNO12,        PARTNO13,        PARTNO14,        PARTNO15, \
        PARTNO16,        PARTNO17,        PARTNO18,        PARTNO19,        PARTNO20, \
        PARTNO21,        PARTNO22,        PARTNO23,        PARTNO24,        PARTNO25, \
        PARTNO26,        PARTNO27,        PARTNO28,        PARTNO29,        PARTNO30, \
        PARTNO31,        PARTNO32,        PARTNO33,        PARTNO34,        PARTNO35, \
        PARTNO36,        PARTNO37,        PARTNO38,        PARTNO39,        PARTNO40, \
        PARTNO41,        PARTNO42,        PARTNO43,        PARTNO44,        PARTNO45, \
        PARTNO46,        PARTNO47,        PARTNO48,        PARTNO49,        PARTNO50, \
        PARTNO51,        PARTNO52,        PARTNO53,        PARTNO54,        PARTNO55, \
        PARTNO56
        ]])[0]

    norm_Area   = (predict_cur_Area   - min_Area  )/(max_Area   - min_Area  )
    norm_Pelvic = (predict_cur_Pelvic - min_Pelvic)/(max_Pelvic - min_Pelvic)
    print(predict_cur_Area, norm_Area)
    # normed_Area_value   = norm(predict_cur_Area)
    # normed_Pelvic_value = norm(predict_cur_Pelvic)
    # np.sqrt(sum((y - predict_cur_Area)**2) / len(y))

    # print( predict_cur )
    # from numpy import linalg as LA
    # val_Area   = LA.norm(predict_cur_Area)
    # val_Pelvic = LA.norm(predict_cur_Pelvic)

    # from sklearn.preprocessing import MinMaxScaler
    # scaler = MinMaxScaler()

    # val_Area   = scaler.fit_transform(predict_cur_Area[:])
    # val_Pelvic = scaler.fit_transform(predict_cur_Pelvic[:])
        
    # print( val_Area )
    # print( val_Pelvic )
    # print( val_Area + val_Pelvic )

    return   predict_cur_Pelvic


In [ ]:
# bnds; doundary conditions
# Parameter of Boundary conditions
bnds = [( 440, 1400 ), ( 440, 1400 ), ( 440, 1400 ), ( 440, 1400 ), ( 440, 1400 ), 
        ( 440, 1400 ), ( 440, 1400 ), ( 440, 1400 ), ( 440, 1400 ), ( 440, 1400 ), 
        ( 440, 1400 ), ( 440, 1400 ), ( 440, 1400 ), ( 440, 1400 ), ( 440, 1400 ), 
        ( 440, 1400 ), ( 440, 1400 ), ( 440, 1400 ), ( 440, 1400 ), ( 440, 1400 ), 
        ( 440, 1400 ), ( 440, 1400 ), ( 440, 1400 ), ( 440, 1400 ), ( 440, 1400 ), 
        ( 440, 1400 ), ( 440, 1400 ), ( 440, 1400 ), ( 440, 1400 ), ( 440, 1400 ), 
        ( 440, 1400 ), ( 440, 1400 ), ( 440, 1400 ), ( 440, 1400 ), ( 440, 1400 ), 
        ( 440, 1400 ), ( 440, 1400 ), ( 440, 1400 ), ( 440, 1400 ), ( 440, 1400 ),                       ( 440, 1400 ), ( 440, 1400 ), ( 440, 1400 ), ( 440, 1400 ), ( 440, 1400 ), 
        ( 440, 1400 ), ( 440, 1400 ), ( 440, 1400 ), ( 440, 1400 ), ( 440, 1400 ),
        ( 440, 1400 ), ( 440, 1400 ), ( 440, 1400 ), ( 440, 1400 ), ( 440, 1400 ),
        ( 440, 1400 )
]
print(len(bnds))

In [468]:
from scipy import optimize
# best1bin
result_best1bin = optimize.differential_evolution(predict_func, bnds, strategy='best1bin', init='latinhypercube')



84] [1.1017094]
[4972.5317] [1.153323]
[4390.776] [1.0739282]
[3785.922] [0.9913811]
[4885.5845] [1.1414571]
[5078.2383] [1.1677493]
[4667.0103] [1.1116272]
[4802.011] [1.1300514]
[4556.611] [1.0965605]
[4437.737] [1.0803373]
[3901.826] [1.007199]
[3745.3293] [0.9858412]
[5108.31] [1.1718534]
[5195.3486] [1.1837319]
[4344.6816] [1.0676376]
[4283.2515] [1.0592539]
[3916.09] [1.0091457]
[4671.1045] [1.112186]
[5186.522] [1.1825273]
[5148.3745] [1.1773212]
[4286.597] [1.0597105]
[4751.272] [1.1231267]
[4425.1313] [1.0786169]
[3686.5146] [0.9778145]
[4710.2485] [1.1175281]
[2589.1316] [0.8280497]
[3954.411] [1.0143756]
[5045.667] [1.1633041]
[4339.4536] [1.0669241]
[4327.8325] [1.0653381]
[4577.6826] [1.0994362]
[4943.1973] [1.1493196]
[4051.239] [1.0275902]
[4234.7573] [1.0526357]
[5050.4995] [1.1639637]
[4877.716] [1.1403831]
[4218.225] [1.0503795]
[3440.632] [0.94425786]
[5301.6426] [1.1982383]
[4754.5854] [1.123579]
[4052.3403] [1.0277405]
[4115.8906] [1.0364133]
[4826.335] [1.1333709]

KeyboardInterrupt: 

In [ ]:
# Get minimize parameters
minimum_best1bin_x = result_best1bin.x
minimum_best1bin_f = result_best1bin.fun
minimum_best1bin_x, minimum_best1bin_f

In [ ]:
from scipy import optimize
# best1bin popsize 20
result_best1bin = optimize.differential_evolution(predict_func, bnds, popsize=20, strategy='best1bin', init='latinhypercube')

In [ ]:
minimum_best1bin_x = result_best1bin.x
minimum_best1bin_f = result_best1bin.fun
minimum_best1bin_x, minimum_best1bin_f

In [ ]:
from scipy import optimize
# best1bin popsize 50
result_best1bin = optimize.differential_evolution(predict_func, bnds, popsize=50, strategy='best1bin', init='latinhypercube')


In [ ]:
# Get minimize parameters
minimum_best1bin_x = result_best1bin.x
minimum_best1bin_f = result_best1bin.fun
minimum_best1bin_x, minimum_best1bin_f

In [ ]:
# best1exp
result_best1exp = optimize.differential_evolution(predict_func, bnds, strategy=‘best1exp’, init='latinhypercube',)

# Get minimize parameters
minimum = result_genetic.x
minimum

In [ ]:
# Import Keras training result
Area_MODEL = load_model("{}/Thorax/SAB_linear_thorax_train_80p_200612_10var.h5".format(current_dir))
Disp_MODEL = load_model("{}/Pelvis/SAB_linear_pelvis_train_80p_200612_10var.h5".format(current_dir))

In [ ]:
def predict_func(Parameter):
    Area_value =  np.sum( (T_Rforc[:T_point ] - T_predict_cur[: T_point ])**2 ) / len(T_Rforc) 
    Disp_value =  np.sum( (P_Rforc[:P_point ] - P_predict_cur[: P_point ])**2 ) / len(P_Rforc)     
    return   Area_value + Disp_value